In [21]:
import sqlite3
import csv
import requests
import time
from bs4 import BeautifulSoup
from datetime import datetime
import os
import logging
import tempfile
import random
from tqdm import tqdm
import io 

In [22]:
# Configuration du logging
logging.basicConfig(
    filename="ultimate-db.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)

AUTHORIZED_TYPES = ["game", "dlc", 'demo', 'beta', '']
BATCH_SIZE = 10000
LOG_FREQ = 2000
COMMIT_FREQUENCY = 500

In [23]:
def create_database():
    conn = sqlite3.connect('all-steampages-data.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS steam_games (
        game_id INTEGER PRIMARY KEY,
        add_date INTEGER,
        type TEXT,
        dev TEXT,
        publisher TEXT,
        release_date INTEGER,
        description TEXT,
        nb_reviews INTEGER,
        free INTEGER,
        dlc INTEGER,
        dlc_list TEXT,
        price TEXT,
        metacritic INTEGER,
        genres TEXT,
        singleplayer INTEGER,
        multiplayer INTEGER,
        coop INTEGER,
        online_coop INTEGER,
        lan_coop INTEGER,
        shared_split_screen_coop INTEGER,
        shared_split_screen INTEGER,
        pvp INTEGER,
        lan_pvp INTEGER,
        shared_split_screen_pvp INTEGER,
        achievements INTEGER,
        full_controller_support INTEGER,
        trading_cards INTEGER,
        steam_cloud INTEGER,
        remote_play_phone INTEGER,
        remote_play_tablet INTEGER,
        remote_play_together INTEGER,
        remote_play_tv INTEGER,
        family_sharing INTEGER,
        captions_available INTEGER,
        inapp_purchases INTEGER,
        early_access INTEGER,
        vr_only INTEGER,
        vr_supported INTEGER,
        online_pvp INTEGER,
        required_age INTEGER,
        controller_support TEXT,
        categories TEXT,
        website TEXT,
        support_mail TEXT,
        support_url TEXT,
        cd_some_nudity_or_sexual_content INTEGER,
        cd_frequent_violence_gore INTEGER,
        cd_adult_only_sexual_content INTEGER,
        cd_frequent_nudity_or_sexual_content INTEGER,
        cd_general_mature_content INTEGER,
        lg_en INTEGER,
        lg_ger INTEGER,
        lg_spa INTEGER,
        lg_jap INTEGER,
        lg_portuguese INTEGER,
        lg_russian INTEGER,
        lg_simp_chin INTEGER,
        lg_trad_chin INTEGER,
        lg_fr INTEGER,
        lg_it INTEGER,
        lg_hung INTEGER,
        lg_kor INTEGER,
        lg_turk INTEGER,
        lg_arabic INTEGER,
        lg_polish INTEGER,
        lg_thai INTEGER,
        lg_viet INTEGER
    )
    ''')

    conn.commit()
    return conn

def create_database_excluded():
    conn = sqlite3.connect('excluded_games.db')
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS excluded_games (
        game_id INTEGER PRIMARY KEY
    )
    ''')
    conn.commit()
    return conn

def get_steam_data(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]['success']:
            return data[str(app_id)]['data']
    return None

def parse_release_date(date_str):
    if date_str in ["Coming soon", "To be announced"]:
        return None
    try:
        if "Q" in date_str:
            year = int(date_str.split()[-1])
            quarter = int(date_str[1])
            month = (quarter - 1) * 3 + 1
            return int(datetime(year, month, 1).timestamp())
        elif len(date_str.split()) == 2:
            return int(datetime.strptime(f"1 {date_str}", "%d %B %Y").timestamp())
        else:
            return int(datetime.strptime(date_str, "%d %b, %Y").timestamp())
    except:
        return None

def insert_game_data(cursor: sqlite3.Cursor, game_data: dict):
    # Extraction et transformation des données
    game_id = game_data['steam_appid']
    add_date = int(time.time())
    game_type = game_data.get('type', '')
    dev = ', '.join(game_data.get('developers', []))
    publisher = ', '.join(game_data.get('publishers', []))
    release_date = parse_release_date(game_data.get('release_date', {}).get('date', ''))
    description = game_data.get('short_description', '')
    nb_reviews = game_data.get('recommendations', {}).get('total', 0)
    free = 1 if game_data.get('is_free', False) else 0
    dlc = 1 if game_type == 'dlc' else 0
    dlc_list = ','.join(map(str, game_data.get('dlc', [])))
    price = game_data.get('price_overview', {}).get('final_formatted', '')
    metacritic = game_data.get('metacritic', {}).get('score', None)
    genres = ','.join([genre['description'] for genre in game_data.get('genres', [])])
    
    categories = game_data.get('categories', [])
    category_ids = [cat['id'] for cat in categories]
    
    singleplayer = 1 if 2 in category_ids else 0
    multiplayer = 1 if 1 in category_ids else 0
    coop = 1 if 9 in category_ids else 0
    online_coop = 1 if 38 in category_ids else 0
    lan_coop = 1 if 48 in category_ids else 0
    shared_split_screen_coop = 1 if 39 in category_ids else 0
    shared_split_screen = 1 if 24 in category_ids else 0
    pvp = 1 if 49 in category_ids else 0
    lan_pvp = 1 if 47 in category_ids else 0
    shared_split_screen_pvp = 1 if 37 in category_ids else 0
    achievements = 1 if 22 in category_ids else 0
    full_controller_support = 1 if 28 in category_ids else 0
    trading_cards = 1 if 29 in category_ids else 0
    steam_cloud = 1 if 23 in category_ids else 0
    remote_play_phone = 1 if 41 in category_ids else 0
    remote_play_tablet = 1 if 42 in category_ids else 0
    remote_play_together = 1 if 44 in category_ids else 0
    remote_play_tv = 1 if 43 in category_ids else 0
    family_sharing = 1 if 62 in category_ids else 0
    captions_available = 1 if 13 in category_ids else 0
    inapp_purchases = 1 if 35 in category_ids else 0
    
    early_access = 1 if '70' in [genre['id'] for genre in game_data.get('genres', [])] else 0
    vr_only = 1 if 54 in category_ids else 0
    vr_supported = 1 if 53 in category_ids else 0
    online_pvp = 1 if 36 in category_ids else 0
    
    required_age = game_data.get('required_age', 0)
    controller_support = game_data.get('controller_support', '')
    website = game_data.get('website', '')
    categories_str = ','.join([cat['description'] for cat in categories])
    support_info = game_data.get('support_info', {})
    support_mail = support_info.get('email', '')
    support_url = support_info.get('url', '')
    
    content_descriptors = game_data.get('content_descriptors', {}).get('ids', [])
    cd_some_nudity_or_sexual_content = 1 if 1 in content_descriptors else 0
    cd_frequent_violence_gore = 1 if 2 in content_descriptors else 0
    cd_adult_only_sexual_content = 1 if 3 in content_descriptors else 0
    cd_frequent_nudity_or_sexual_content = 1 if 4 in content_descriptors else 0
    cd_general_mature_content = 1 if 5 in content_descriptors else 0
    
    supported_languages = game_data.get('supported_languages', '')
    lg_en = 1 if 'English' in supported_languages else 0
    lg_ger = 1 if 'German' in supported_languages else 0
    lg_spa = 1 if 'Spanish - Spain' in supported_languages else 0
    lg_jap = 1 if 'Japanese' in supported_languages else 0
    lg_portuguese = 1 if 'Portuguese - Brazil' in supported_languages else 0
    lg_russian = 1 if 'Russian' in supported_languages else 0
    lg_simp_chin = 1 if 'Simplified Chinese' in supported_languages else 0
    lg_trad_chin = 1 if 'Traditional Chinese' in supported_languages else 0
    lg_fr = 1 if 'French' in supported_languages else 0
    lg_it = 1 if 'Italian' in supported_languages else 0
    lg_hung = 1 if 'Hungarian' in supported_languages else 0
    lg_kor = 1 if 'Korean' in supported_languages else 0
    lg_turk = 1 if 'Turkish' in supported_languages else 0
    lg_arabic = 1 if 'Arabic' in supported_languages else 0
    lg_polish = 1 if 'Polish' in supported_languages else 0
    lg_thai = 1 if 'Thai' in supported_languages else 0
    lg_viet = 1 if 'Vietnamese' in supported_languages else 0
    
    # Insertion des données dans la base de données
    cursor.execute('''
    INSERT INTO steam_games (
        game_id, add_date, type, dev, publisher, release_date, description, nb_reviews,
        free, dlc, dlc_list, price, metacritic, genres, singleplayer, multiplayer,
        coop, online_coop, lan_coop, shared_split_screen_coop, shared_split_screen,
        pvp, lan_pvp, shared_split_screen_pvp, achievements, full_controller_support,
        trading_cards, steam_cloud, remote_play_phone, remote_play_tablet,
        remote_play_together, remote_play_tv, family_sharing, captions_available,
        inapp_purchases, early_access, vr_only, vr_supported, online_pvp,
        required_age, controller_support, categories, website, support_mail,
        support_url, cd_some_nudity_or_sexual_content,
        cd_frequent_violence_gore, cd_adult_only_sexual_content,
        cd_frequent_nudity_or_sexual_content, cd_general_mature_content,
        lg_en, lg_ger, lg_spa, lg_jap, lg_portuguese, lg_russian, lg_simp_chin,
        lg_trad_chin, lg_fr, lg_it, lg_hung, lg_kor, lg_turk, lg_arabic,
        lg_polish, lg_thai, lg_viet
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
              ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, 
              ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (game_id, add_date, game_type, dev, publisher, release_date, description, nb_reviews,
        free, dlc, dlc_list, price, metacritic, genres, singleplayer, multiplayer,
        coop, online_coop, lan_coop, shared_split_screen_coop, shared_split_screen,
        pvp, lan_pvp, shared_split_screen_pvp, achievements, full_controller_support,
        trading_cards, steam_cloud, remote_play_phone, remote_play_tablet,
        remote_play_together, remote_play_tv, family_sharing, captions_available,
        inapp_purchases, early_access, vr_only, vr_supported, online_pvp,
        required_age, controller_support, categories_str, website, support_mail,
        support_url, cd_some_nudity_or_sexual_content,
        cd_frequent_violence_gore, cd_adult_only_sexual_content,
        cd_frequent_nudity_or_sexual_content, cd_general_mature_content,
        lg_en, lg_ger, lg_spa, lg_jap, lg_portuguese, lg_russian, lg_simp_chin,
        lg_trad_chin, lg_fr, lg_it, lg_hung, lg_kor, lg_turk, lg_arabic,
        lg_polish, lg_thai, lg_viet
    ))

    return True

def get_game_ids_to_process(cursor: sqlite3.Cursor) -> list[int]:
    """Get the list of game ids to process"""
    # Get all game_id that are not in game_reviews
    res = cursor.execute(
        f"""
            SELECT steam_game_id FROM games
            WHERE steam_game_id NOT IN (
                         SELECT game_id FROM steam_games
            )
            AND steam_game_id NOT IN (
                         SELECT game_id FROM excluded_games
            )
            ORDER BY steam_game_id ASC 
            """
    ).fetchall()
    games_id = [row[0] for row in res]
    return games_id

def log_excluded_games(cursor: sqlite3.Cursor, game_id: int):
    try:
        cursor.execute(
            """ 
            INSERT OR REPLACE INTO excluded_games (
                game_id)
                VALUES (?)
            """,
            (game_id,)
        )
        return True
    except Exception as e:
        logging.error(
            f"Unexpected error when logging game_id into excluded_games : {game_id}: {e}"
        )

    return False

def csv_to_sqlite_temp(csv_file_path):
    # Créer un fichier temporaire pour la base de données SQLite
    temp_db = tempfile.NamedTemporaryFile(suffix='.db', delete=False)
    db_path = temp_db.name
    temp_db.close()

    # Connexion à la base de données SQLite temporaire
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Créer la table
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS games (
        id INTEGER PRIMARY KEY,
        steam_game_id INTEGER,
        first_seen INTEGER
    )
    ''')

    # Vérifier si le chemin est une URL
    if csv_file_path.startswith('http://') or csv_file_path.startswith('https://'):
        # Si c'est une URL, télécharger le contenu
        response = requests.get(csv_file_path)
        response.raise_for_status()  # Cela va lever une exception pour les codes d'état HTTP 4XX/5XX
        csv_content = io.StringIO(response.text)
    else:
        # Si c'est un fichier local, l'ouvrir normalement
        csv_content = open(csv_file_path, 'r', newline='')

    # Lire le contenu CSV et insérer les données dans la base de données
    csvreader = csv.DictReader(csv_content)
    for row in csvreader:
        cursor.execute('''
        INSERT INTO games (id, steam_game_id, first_seen)
        VALUES (?, ?, ?)
        ''', (int(row['id']), int(row['steam_game_id']), int(row['first_seen'])))

    # Fermer le fichier si c'était un fichier local
    if not isinstance(csv_content, io.StringIO):
        csv_content.close()

    # Commit les changements et fermer la connexion
    conn.commit()
    conn.close()

    return db_path

In [24]:
def main():
    total_inserted = 0
    
    # Créer ou se connecter à la base de données ultime
    ultimate_conn = create_database()
    
    # Télécharger le fichier CSV des jeux Steam
    temp_db_path = csv_to_sqlite_temp("https://raw.githubusercontent.com/belzanne/steampage-creation-date/main/steam_games.csv")
    print(f"Base de données temporaire créée à : {temp_db_path}")
    excluded_conn = create_database_excluded()

    # Attach the Steam games database to the game_reviews.db
    ultimate_conn.execute(f"ATTACH '{temp_db_path}' AS games")
    ultimate_conn.execute(f"ATTACH 'excluded_games.db' AS excluded_games")
    
    # Get game_ids to process
    games_ids = get_game_ids_to_process(ultimate_conn)
    tot_games = len(games_ids)
    games_ids = games_ids[:BATCH_SIZE]
    logging.info(
        f"""{len(games_ids)} / {tot_games} games to process.
    ⓘ Increase BATCH_SIZE to process more games"""
    )

    print(games_ids)

    try:
        for i, game_id in enumerate(tqdm(games_ids)):
            id_data = get_steam_data(game_id)

            if id_data: 

                if id_data['type'] in AUTHORIZED_TYPES:
                    try:
                        insert_game_data(ultimate_conn, id_data)
                        total_inserted += 1
                        if total_inserted % COMMIT_FREQUENCY == 0:
                            ultimate_conn.commit()
                            logging.info(f'{total_inserted} jeux insérés et commités')
                        #logging.info(f'game_id inséré dans all-steampages-data.db : {game_id}')
                    except Exception as e:
                        logging.info("erreur lors de l'inseration de l'id_data de {game_id} dans ultimate")

                else:
                     #logging.info(f'Unauthorized type for {id_data}')
                     log_excluded_games(ultimate_conn, game_id)
                     ultimate_conn.commit()

            else:
                #logging.info(f'No id_data for {game_id}')
                log_excluded_games(ultimate_conn, game_id)
                ultimate_conn.commit()
                
            if i % LOG_FREQ == 0:
                ultimate_conn.commit()
                logging.info(
                    f"{i} jeux traités. Dernier game_id: {game_id}. Total reviews insérées: {total_inserted}"
                )

            # Ajouter un délai aléatoire entre les requêtes
            time.sleep(random.uniform(0.3, 1.2))

    except Exception as e:
        logging.error(f"Unexpected error: {e}")

    finally:
        if "ultimate_conn" in locals():
            ultimate_conn.close()
            logging.info("Database connection closed.")


In [26]:
if __name__ == "__main__":
    main()

Base de données temporaire créée à : /var/folders/b5/ckj362xj7ng_xqbjrqp077qh0000gn/T/tmpw8w1db23.db
[100, 659, 2130, 2525, 2545, 2550, 4240, 6129, 10190, 12370, 12380, 17340, 18450, 18460, 21110, 21120, 22360, 24043, 24049, 24058, 24085, 31109, 31160, 34440, 34450, 34460, 34470, 35010, 35725, 37420, 39360, 42710, 47830, 47889, 47936, 48173, 52003, 56437, 99830, 201270, 202990, 209170, 209660, 213176, 223591, 234040, 265490, 300220, 300260, 300280, 300300, 300320, 300340, 300360, 300380, 300400, 300440, 300441, 300442, 300450, 300460, 300510, 300540, 300550, 300570, 300580, 300600, 300610, 300620, 300630, 300640, 300641, 300642, 300660, 300661, 300662, 300663, 300730, 300780, 300790, 300820, 300840, 300860, 300870, 300900, 300910, 300920, 300930, 300960, 300970, 300980, 300981, 300982, 300990, 300991, 300992, 301050, 301060, 301110, 301111, 301112, 301113, 301114, 301115, 301116, 301120, 301140, 301150, 301170, 301180, 301190, 301200, 301210, 301220, 301240, 301241, 301242, 301243, 301

 62%|██████▏   | 6223/10000 [1:45:28<1:04:01,  1.02s/it]


KeyboardInterrupt: 